In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import sys

#from sklearn.tree import 
from scipy.sparse import coo_matrix

from sklearn.metrics.pairwise import cosine_similarity

import warnings

import timeit

import math
import operator

print ('All the required modules are imported')

All the required modules are imported


In [2]:
start_time=timeit.default_timer()

data_f = pd.read_csv('bitbootcamp/ml-1m/ratings.dat',sep='::',names=['userID','movieID','rating','timestamp'])
elapsed_time = timeit.default_timer() - start_time
print elapsed_time

C:\Users\Robert\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


5.17024976255


In [3]:
data_f.isnull().sum()

userID       0
movieID      0
rating       0
timestamp    0
dtype: int64

In [4]:
print len(data_f['userID'].unique())
print len(data_f['movieID'].unique())

6040
3706


In [5]:
start_time=timeit.default_timer()
userID, movieID,rating=np.loadtxt('bitbootcamp/ml-1m/ratings.dat',delimiter='::',skiprows=0,usecols=(0,1,2),unpack=True)
elapsed_time = timeit.default_timer() - start_time
print elapsed_time

userID = userID.astype(int)
movieID = movieID.astype(int)

unique_movies = list(np.unique(movieID))
unique_movies[:5]

6.06076701388


[1, 2, 3, 4, 5]

In [6]:
print list(data_f.keys())

['userID', 'movieID', 'rating', 'timestamp']


In [7]:
data_f.describe()

,userID,movieID,rating,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [8]:
#This puts the ith element of np.ones(len(userID)) in the (movieID,userID)th place of the sparse matrix
#We want to be able to index by row when we extract to create the cosine similarity matrix
user_movie = coo_matrix((rating,(movieID,userID)))

In [23]:
file_path = 'bitbootcamp/ml-1m/movies.dat'
mdict={}

for line in open(file_path,'r'):
    L = line.strip('\n').split('::')
    #Translate special characters
    L[1] = L[1].decode('latin-1')
    #Removes years and translations
    L[1] = L[1].split(' (')[0]
    #Place preposition at beginning of title
    #might have missed some, but didn't want to split by comma since some have comma in title
    if L[1][-3:] == 'The':
        L[1] = 'The ' + L[1][:-5]
    if L[1][-1] == 'A':
        L[1] = 'A' + L[1][:-3]
    if L[1][-2:] == 'An':
        L[1] = 'An' + L[1][:-4]
    if L[1][-3:] == 'Les':
        L[1] = 'Les ' + L[1][:-5]
    if L[1][-2:] == "L'":
        L[1] = "L'" + L[1][:-4]
    if L[1][-2:] == "Le":
        L[1] = "Le " + L[1][:-4]
    #Slower but helps to solve issue with multiple years, same title
    if L[1].lower() in mdict.keys():
        mdict[L[1].lower()].append(int(L[0]))
    else:
        mdict[L[1].lower()] = [int(L[0])]

In [24]:
#Just to check
#mdict

{u'jeanne and the perfect guy': [2591],
 u'aking in new york': [3640],
 u'my fellow americans': [1390],
 u'toy story 2': [3114],
 u'blood & wine': [1351],
 u'two family house': [3951],
 u'vermin': [1738],
 u'the trip to bountiful': [2069],
 u'andre': [577],
 u'the haunted world of edward d. wood jr.': [722],
 u'prisoner of the mountains': [1450],
 u"freddy's dead: the final nightmare": [1973],
 u'mirage': [401],
 u'basic instinct': [1092],
 u'grosse pointe blank': [1500],
 u'shadow of angels': [873],
 u'hate': [97],
 u'goldfinger': [2947],
 u'jungle fever': [3426],
 u"akid in king arthur's court": [258],
 u'heartbreak ridge': [2476],
 u'buddy': [1551],
 u'for a few dollars more': [3681],
 u'get shorty': [21],
 u'the edge': [1615],
 u'the transformers: the movie': [1205],
 u'soapdish': [3712],
 u"wes craven's new nightmare": [366],
 u'human traffic': [3581],
 u'the brothers mcmullen': [144],
 u'blame it on rio': [2259],
 u'highlander iii: the sorcerer': [405],
 u'the bat': [2814],
 u'de

In [25]:
#Get the id's for the movies (if there were multiple years, add both id's on)
cos_sim_id=[]
while 1:
    val = raw_input('Enter the movie name: ')
    if val=='exit':
        sys.exit(0)
    cos_sim_id+=mdict[val.lower()]

Enter the movie name: the man who knew too much
Enter the movie name: the godfather
Enter the movie name: exit


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


In [26]:
#Get the cosine_similarity matrix by using the sparse matrix and a splice of it
start = timeit.default_timer()
csm_params = {'X': user_movie,
              'Y': user_movie.tocsr()[cos_sim_id]}
cos = cosine_similarity(**csm_params)
elapsed = timeit.default_timer() - start
print elapsed


0.0945229972872


In [28]:
#Reverse dictionary to retrieve titles from ids
mdictrev = {}
for key in mdict:
    for value in mdict[key]:
        mdictrev[value] = key

In [29]:
#Dictionary of all titles and their respective cosine similarities
dic = {}
for idx,list in enumerate(cos):
    #We don't want those already in list
    if idx not in cos_sim_id:
        #Also disregard those with no similarity (including those without movies)
        s = sum(list)
        if s != 0.:
            dic[mdictrev[idx]] = sum(list)


In [41]:
#Sort by value
sorted_dic = sorted(dic.items(), key=operator.itemgetter(1),reverse=True)

In [42]:
#Just to check
#sorted_dic

[(u'the godfather: part ii', 1.1457737934533041),
 (u'vertigo', 1.0828663098748177),
 (u'rear window', 1.0827834366778615),
 (u'dial m for murder', 1.0175966023856022),
 (u'citizen kane', 0.99816059262349643),
 (u'north by northwest', 0.99764289102541215),
 (u'the french connection', 0.9959655516821676),
 (u'chinatown', 0.99356741688697281),
 (u'strangers on a train', 0.99162622555988877),
 (u'casablanca', 0.98723701509138651),
 (u'apocalypse now', 0.97487670828782969),
 (u'star wars: episode iv - a new hope', 0.96594469833597718),
 (u'the wizard of oz', 0.96162886347683463),
 (u'the graduate', 0.95816691108155561),
 (u'fargo', 0.95721771320722582),
 (u'star wars: episode v - the empire strikes back', 0.950647560019242),
 (u'alien', 0.94780209414601724),
 (u'taxi driver', 0.94719724440931785),
 (u'goldfinger', 0.94168845066293883),
 (u'raiders of the lost ark', 0.9384688563417336),
 (u'dr. strangelove or: how i learned to stop worrying and love the bomb',
  0.93538096489868516),
 (u'no

In [43]:
#This might print multiple of the same title if there are different years
for i in range(10):
    print 'Recommendation '+str(i+1)+': '+sorted_dic.pop(0)[0]

Recommendation 1: the godfather: part ii
Recommendation 2: vertigo
Recommendation 3: rear window
Recommendation 4: dial m for murder
Recommendation 5: citizen kane
Recommendation 6: north by northwest
Recommendation 7: the french connection
Recommendation 8: chinatown
Recommendation 9: strangers on a train
Recommendation 10: casablanca
